# 1 Pandas 数值运算方法
对于一元运算（像函数与三角函数），这些通用函
数将在输出结果中保留索引和列标签；而对于二元运算（如加法和乘法），Pandas 在传递
通用函数时会自动对齐索引进行计算。这就意味着，保存数据内容与组合不同来源的数
据——两处在 NumPy 数组中都容易出错的地方——变成了 Pandas 的杀手锏。后面还会介
绍一些关于一维 Series 和二维 DataFrame 的便捷运算方法

### 1.1通用函数 ： 保留索引

In [1]:
import pandas as pd
import numpy as np
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [2]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


如果对这两个对象的其中一个使用 NumPy 通用函数，生成的结果是另一个保留索引的
Pandas 对象：

In [3]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

In [4]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


### 1.2通用函数 ： 索引对齐

当在两个 Series 或 DataFrame 对象上进行二元计算时，Pandas 会在计算过程中对齐两个对
象的索引

##### 1.2.1. Series 索引对齐

In [7]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
'New York': 19651127}, name='population')
print(population / area)
print(area.index | population.index)

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64
Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')


/tmp/ipykernel_45240/1288574611.py:6: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead
  print(area.index | population.index)


In [9]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
print(A + B)
# 如果用 NaN 值不是我们想要的结果，那么可以用适当的对象方法代替运算符。例如，
# A.add(B) 等价于 A + B ，也可以设置参数自定义 A 或 B 缺失的数据：
print(A.add(B, fill_value=0))

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64
0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64


##### 1.2.2. DataFrame 索引对齐
在计算两个 DataFrame 时，类似的索引对齐规则也同样会出现在共同（并集）列中

In [11]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
columns=list('AB'))
print(A)
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
columns=list('BAC'))
print(B)
print(A + B)

    A   B
0   0  11
1  11  16
   B  A  C
0  9  2  6
1  3  8  2
2  4  2  6
      A     B   C
0   2.0  20.0 NaN
1  19.0  19.0 NaN
2   NaN   NaN NaN


两个对象的行列索引可以是不同顺序的，结果的索引会自动按顺序排列。在
Series 中，我们可以通过运算符方法的 fill_value 参数自定义缺失值。这里，我们将用 A
中所有值的均值来填充缺失值（计算 A 的均值需要用 stack 将二维数组压缩成一维数组）

### 1.2.3 通用函数 ： DataFrame 与 Series 的运算

In [15]:
A = rng.randint(10, size=(3, 4))
print(A)
print(A - A[0])
df = pd.DataFrame(A, columns=list('QRST'))
# 让二维数组减自身的一行数据会按行计算
print(df - df.iloc[0])
# 如果你想按列计算，那么就需要利用前面介绍过的运算符方法，通过 axis 参数设置：
print(df.subtract(df['R'], axis=0))

halfrow = df.iloc[0, ::2]
print(halfrow)
print(df - halfrow)

[[4 1 4 7]
 [9 8 8 0]
 [8 6 8 7]]
[[ 0  0  0  0]
 [ 5  7  4 -7]
 [ 4  5  4  0]]
   Q  R  S  T
0  0  0  0  0
1  5  7  4 -7
2  4  5  4  0
   Q  R  S  T
0  3  0  3  6
1  1  0  0 -8
2  2  0  2  1
Q    4
S    4
Name: 0, dtype: int64
     Q   R    S   T
0  0.0 NaN  0.0 NaN
1  5.0 NaN  4.0 NaN
2  4.0 NaN  4.0 NaN


# 2. 处理缺失值
缺失值主要有三种形
式： null 、 NaN 或 NA

### 2.1 选择处理缺失值的方法

一般情况下可以分为两种：一种方法
是通过 **一个覆盖全局的掩码表示缺失值**，另一种方法是用一个 **标签值（sentinel value）表
示缺失值** 。
+ 在掩码方法中，掩码可能是一个与原数组维度相同的完整布尔类型数组，也可能是用一个
比特（0 或 1）表示有缺失值的局部状态。
，也可能是些
+ 在标签方法中，标签值可能是具体的数据（例如用 - 9999 表示缺失的整数）
极少出现的形式。另外，标签值还可能是更全局的值，比如用 NaN （不是一个数）表示缺
失的浮点数，它是 IEEE 浮点数规范中指定的特殊字符。

### 2.2 Pandas 的缺失值
##### 2.2.1. None ：Python 对象类型的缺失值
Pandas 可以使用的第一种缺失值标签是 None ，它是一个 Python 单体对象，经常在代码中
表示缺失值。由于 None 是一个 Python 对象，所以不能作为任何 NumPy / Pandas 数组类型
的缺失值，只能用于 'object' 数组类型（即由 Python 对象构成的数组）

In [16]:
import numpy as np
import pandas as pd

vals1 = np.array([1, None, 3, 4])
print(vals1)

for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

[1 None 3 4]
dtype = object
29.7 ms ± 342 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
752 µs ± 6.93 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)



##### 2.2.2. NaN ： 数值类型的缺失值
另一种缺失值的标签是 NaN （全称 Not a Number，不是一个数字），是一种按照 IEEE 浮点
数标准设计、在任何系统中都兼容的特殊浮点数：

In [21]:
vals2 = np.array([1, np.nan, 3, 4])
print(vals2.dtype)
print(1 + np.nan)
print(0 *np.nan)
# 虽然这些累计操作的结果定义是合理的（即不会抛出异常），但是并非总是有效的：
print(vals2.sum(), vals2.min(), vals2.max())
# NumPy 也提供了一些特殊的累计函数，它们可以忽略缺失值的影响：
print(np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2))

float64
nan
nan
nan nan nan
8.0 1.0 4.0


##### 2.2.3. Pandas中 NaN 与 None 的差异
虽然 NaN 与 None 各有各的用处，但是 Pandas 把它们看成是可以等价交换的，在适当的时
候会将两者进行替换：

In [22]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Pandas 会将没有标签值的数据类型自动转换为 NA 。例如，当我们将整型数组中的一个值设
置为 np.nan 时，这个值就会强制转换成浮点数缺失值 NA 。

In [23]:
x = pd.Series(range(2), dtype=int)
print(x)
x[0] = None
print(x)

0    0
1    1
dtype: int64
0    NaN
1    1.0
dtype: float64


### 2.3处理缺失值
+ isnull()
创建一个布尔类型的掩码标签缺失值。

+ notnull()
与 isnull() 操作相反。

+ dropna()
返回一个剔除缺失值的数据。

+ fillna()
返回一个填充了缺失值的数据副本。

##### 2.3.1. 发现缺失值
Pandas 数据结构有两种有效的方法可以发现缺失值： isnull() 和 notnull() 。每种方法都
返回布尔类型的掩码数据

In [24]:
data = pd.Series([1, np.nan, 'hello', None])
print(data.isnull())
print(data[data.notnull()])

0    False
1     True
2    False
3     True
dtype: bool

In [25]:
# 布尔类型掩码数组可以直接作为 Series 或 DataFrame 的索引使用
data[data.notnull()]

0        1
2    hello
dtype: object

##### 2.3.2. 剔除缺失值
两种很好用的缺失值处理方法，分别是 dropna() （剔除缺
失值）和 fillna() （填充缺失值）

In [26]:
data.dropna()

0        1
2    hello
dtype: object

In [28]:
df = pd.DataFrame([[1,
np.nan, 2],
[2,
3,
5],
[np.nan, 4,
6]])
print(df.dropna())
print(df.dropna(axis='columns'))

     0    1  2
1  2.0  3.0  5
   2
0  2
1  5
2  6


In [31]:
df[3] = np.nan
print(df.dropna(axis='columns', how='all'))
print(df.dropna(axis='rows', thresh=3))

     0    1  2
0  1.0  NaN  2
1  2.0  3.0  5
2  NaN  4.0  6
     0    1  2   3
1  2.0  3.0  5 NaN


##### 2.3.3. 填充缺失值
有时候你可能并不想移除缺失值，而是想把它们替换成有效的数值。有效的值可能是像
0、1、2 那样单独的值，也可能是经过填充（imputation）或转换（interpolation）得到的。
虽然你可以通过 isnull() 方法建立掩码来填充缺失值，但是 Pandas 为此专门提供了一个
fillna() 方法，它将返回填充了缺失值后的数组副本

In [35]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
print(data)
print(data.fillna(0))
print(data.fillna(method='ffill'))
print(data.fillna(method='bfill'))
print(df.fillna(method='ffill', axis=1))

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64
a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64
a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64
